In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import missingno as mn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
ss = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
mn.matrix(train)

Let's see how ther target variablei is distributed

In [ ]:
sns.distplot(train.congestion)

we can see that the data is a bit skewed on the right andmost of our data lies between 20 and 80

In [ ]:
train.head()

In [ ]:
def process_time(df):
    
    df['time'] = pd.to_datetime(df['time'])
    df["Month_Number"] = df["time"].dt.month
    df["Week_Day"] = df["time"].dt.dayofweek
    df['year'] = df["time"].dt.year
    df['day'] = df["time"].dt.day
    df['dayofyear'] = df["time"].dt.dayofyear
    df['year'] = df["time"].dt.year
    df['hour'] = df["time"].dt.hour
    df['quarter'] = df["time"].dt.quarter
    df['dayofyear'] = df["time"].dt.dayofyear
#     df['weekend'] = df['time'].dt.dayofweek >=5
#     df['Month_name'] = pd.to_datetime(df['time'], format='%m').dt.month_name().str.slice(stop=3)

    return df

train = process_time(train)
# same thing for test data 
test = process_time(test)

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
test.shape, train.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X = train.drop(['time','congestion','row_id'],axis=1)
y = train.congestion

In [ ]:
test = test.drop(['time','row_id'],axis=1)
test.shape

In [ ]:
X.shape

In [ ]:
test.direction.value_counts().plot(kind='bar')

In [ ]:
train.direction.value_counts().plot(kind='bar')

In [ ]:
X = pd.get_dummies(X, drop_first=True)
X.shape

In [ ]:
# encoding the dicrection

test = pd.get_dummies(test, drop_first=True)
test.shape

In [ ]:
X.info(), test.info()

In [ ]:
X.shape,test.shape

In [ ]:
X.head()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.1,random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
xgb = XGBRegressor(learning_rate=0.01,task_type="GPU",)
xgb.fit(X_train,y_train)
pred = xgb.predict(X_test)
mean_absolute_error(y_test,pred)

In [ ]:
pred_test = xgb.predict(test)

sub = pd.DataFrame({'row_id':ss.row_id, 'congestion':pred_test})
sub.to_csv('xgb.csv',index=False)


In [ ]:
pd.read_csv('xgb.csv')

In [ ]:
target = train.congestion

In [ ]:
X

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold

model = CatBoostRegressor(loss_function="RMSE",
                           eval_metric="MAE",
                           task_type="GPU",
                           learning_rate=0.01,
                           iterations=70000,
                           l2_leaf_reg=50,
                           random_seed=432013,
                
                         
                           early_stopping_rounds=150,
                           border_count=64
                          )
model.fit(X_train,y_train)

In [ ]:
pred_cb = model.predict(X_test)
mean_absolute_error(y_test, pred_cb)

In [ ]:
pred = model.predict(test)

sub = pd.DataFrame({'row_id':ss.row_id, 'congestion':pred})
sub.to_csv('sub_cb.csv',index=False)

In [ ]:
pd.read_csv('sub_cb.csv')